# Scenario Context
Scenario Contexts enables the user to price and calculated risk under varying market states and pricing environments.

In [ ]:
from gs_quant.risk import MarketDataPattern, MarketDataShock, MarketDataShockType, MarketDataShockBasedScenario, \
    CarryScenario
from gs_quant.instrument import IRSwaption
from gs_quant.session import GsSession

client_id = None
client_secret = None

GsSession.use(client_id=client_id, client_secret=client_secret,
              scopes=['run_analytics', 'read_financial_data'])

In [ ]:
# Create and price a swaption
swaption = IRSwaption('Pay', '5y', 'USD', expirationDate='3m')
base_price = swaption.price()
base_price

### MarketDataShockBasedScenario
Allows the user to create a bespoke market data shock

In [ ]:
# Scenario Creation: Shock all points on the Vol Curve by 1bp
ir_vol_scenario = MarketDataShockBasedScenario(
    shocks={
        MarketDataPattern('IR Vol'): MarketDataShock(MarketDataShockType.Absolute, 1e-4)
    }
)

In [ ]:
# Price swaption under scenario
with ir_vol_scenario:
    scenario_price = swaption.price()

scenario_price

In [ ]:
# Swaption Price Comparison: Base vs. Shocked Vol Curves
diff = scenario_price - base_price
diff

In [ ]:
# Comparing Parallel Bump Scenario w/ Vega
from gs_quant.risk import IRVega
vega = swaption.calc(IRVega).value.sum()
vega

### Carry Scenario 
A predefined scenario used to evolve market data and trades over a period of time

In [ ]:
# Carry Scenario - Shift forward 22 business days (1 month)
base_price = swaption.price()
with CarryScenario(time_shift=22):
    fwd_price = swaption.price()

print('Base price:     {:,.2f}'.format(base_price))
print('Scenario price: {:,.2f}'.format(fwd_price))
print('Diff: {:,.2f}'.format(fwd_price - base_price))